## Install BRAILS++
Before running the following cells, install the latest version of BRAILS++.

In [ ]:
!pip install brails

# Import Required Packages

In [1]:
import numpy as np
from brails.utils import Importer

# Define Location Specific Parameters

In [2]:
LOCATION_NAME = 'San Francisco California'
INVENTORY_OUTPUT = 'SFInventory_EQ.geojson'
NO_POSSIBLE_WORLDS = 1

# Create and Importer object to Pull In Required BRAILS Modules

In [3]:
importer = Importer()

# Define the Region Object for the Region of Interest

In [4]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

# Get Raw NSI Data for the Defined Region

In [5]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)

INFO:numexpr.utils:NumExpr defaulting to 12 threads.



Searching for San Francisco California...
Found San Francisco, California, United States


INFO:root:
Getting National Structure Inventory (NSI) building data for the entered location...



Found a total of 167283 building points in NSI that are within the entered region of interest


# Get FEMA USA Footprints Data for the Defined Region

In [6]:
scraper_class = importer.get_class('OSM_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for San Francisco California...
Found San Francisco, California, United States

Found a total of 159898 building footprints in San Francisco


# Create a Baseline Inventory by Merging NSI Raw Data and USA Structures Footprints

In [7]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)


Getting National Structure Inventory (NSI) building data for the entered location...
Found a total of 147204 building points in NSI that match the footprint data.


# Fill Gaps in the Baseline Inventory by Imputing Missing Values

In [8]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

/Users/jinyanzhao/Desktop/SimCenterBuild/brailsPlusPlusTest/.venv/lib/python3.12/site-packages/brails/imputers/knn_imputer/knn_imputer.py:157: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bldg_properties_df = bldg_properties_df.replace("NA", np.nan, inplace=False)


Missing percentages among 159898 assets
buildingheight: 12.55%
erabuilt: 7.90%
numstories: 7.77%
roofshape: 99.88%
fparea: 7.94%
repaircost: 7.94%
constype: 7.94%
occupancy: 7.94%
found_ht: 7.94%
splitlevel: 7.94%
basement: 27.92%
Primitive imputation done.
Running the main imputation. This may take a while.
Enumerating clusters: 20 among 320
Enumerating clusters: 40 among 320
Enumerating clusters: 60 among 320
Enumerating clusters: 80 among 320
Enumerating clusters: 100 among 320
Enumerating clusters: 120 among 320
Enumerating clusters: 140 among 320
Enumerating clusters: 160 among 320
Enumerating clusters: 180 among 320
Enumerating clusters: 200 among 320
Enumerating clusters: 220 among 320
Enumerating clusters: 240 among 320
Enumerating clusters: 260 among 320
Enumerating clusters: 280 among 320
Enumerating clusters: 300 among 320
Enumerating clusters: 320 among 320
Done imputation. It took 0.17 mins


# Generate Synthetic Income Data for Input into HAZUS Rulesets
This is necessary because income data is currently a required input for the HAZUS rulesets in BRAILS++ 

In [9]:
CA_AVG = 78672  # state average
CA_STD_DEV = CA_AVG*0.5  # 50% cov

# Step 1: Calculate the parameters of the underlying normal distribution
mu = np.log(CA_AVG**2 /
            np.sqrt(CA_STD_DEV**2 + CA_AVG**2))
sigma = np.sqrt(np.log(1 + (CA_STD_DEV**2 / CA_AVG**2)))

# Step 2: Generate the lognormal sample using the parameters of the normal
# distribution
for key, val in imputed_inventory.inventory.items():
    lognormal_sample = np.random.lognormal(
        mean=mu, sigma=sigma, size=NO_POSSIBLE_WORLDS)
    val.add_features({"Income": lognormal_sample[0]})

# Change Keys to Make Them Compatible with R2D 

In [10]:
# The names of NEW keys to be inferred.
STRUCTURE_TYPE_KEY = 'StructureTypeHazus'  # instead of  "constype" from NSI
REPLACEMENT_COST_KEY = 'ReplacementCostHazus'  # instead of NSI "repaircost"

# The names of existing keys to be used as "predictors"
YEAR_BUILT_KEY = 'erabuilt'
OCCUPANCY_CLASS_KEY = 'occupancy'
INCOME_KEY = 'Income'
NUMBER_OF_STORIES_KEY = 'numstories'
PLAN_AREA_KEY = 'fpAreas'
SPLIT_LEVEL_KEY = 'splitlevel'

# Run HAZUS Rulesets to Infer R2D-Required Data

In [11]:
infer_features_for_hazuseq = importer.get_class("HazusInfererEarthquake")
inferer = infer_features_for_hazuseq(input_inventory=imputed_inventory,
                                     n_possible_worlds=NO_POSSIBLE_WORLDS,
                                     yearBuilt_key=YEAR_BUILT_KEY,
                                     occupancyClass_key=OCCUPANCY_CLASS_KEY,
                                     numberOfStories_key=NUMBER_OF_STORIES_KEY,
                                     income_key=INCOME_KEY,
                                     splitLevel_key=SPLIT_LEVEL_KEY,
                                     structureType_key=STRUCTURE_TYPE_KEY,
                                     replacementCost_key=REPLACEMENT_COST_KEY,
                                     planArea_key=PLAN_AREA_KEY,
                                     clean_features=False)
hazus_inferred_inventory = inferer.infer()

>> Step1 : Checking if OccupancyClass (occupancy) exist.
>> Step2-1 : Checking if StructureType (StructureTypeHazus) and ReplacementCost (ReplacementCostHazus) exist
>> Step2-2 : Inferring {'ReplacementCostHazus', 'StructureTypeHazus'}


GarageType info not found in the inventory. Making inference using Hazus 6.
ConstructionClass info not found in the inventory. Making inference using Hazus 6.
Done inference. It took 0.36 mins
>> Step3-1 : Checking if BuildingRise (BuildingRise), DesignLevel (DesignLevel) and FoundationType (FoundationType) exist
>> Step3-2 : Inferring {'DesignLevel', 'FoundationType', 'BuildingRise'}


The feature StructureTypeHazus is missing in many buildings including:  [18, 133, 164, 244, 281, 291, 297, 298, 318, 320]
>> Step4 : Changing feature names to what R2D (pelicun) can recognize
Done inference. It took 0.61 mins


From the warning message, inferring HAZUS StructureType for several provided structural types e.g., West Coast-IND1-mid_rise-pre_1950, were not possible using the inference rulestes in BRAILS++, because some provided structural types do not exist in HAZUS's inventory definition. The failed inference has also leaded to missing DesignLevels in the produced inventory here. Below, imputation is used to estimate the HAZUS structural types that do not exist in HAZUS. And the inferrer is run again to estimate DesignLevel.

# Re-run Imputation to Fill Values That Cannot be Inferred by HAZUS Rulesets

In [12]:
imputer = knn_imputer_class(hazus_inferred_inventory, 
                            n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inferred_inventory_imputed = imputer.impute()

Missing percentages among 159898 assets
lon: 7.94%
lat: 7.94%
fd_id: 7.94%
StructureType: 0.14%
Primitive imputation done.
Running the main imputation. This may take a while.
Enumerating clusters: 20 among 320
Enumerating clusters: 40 among 320
Enumerating clusters: 60 among 320
Enumerating clusters: 80 among 320
Enumerating clusters: 100 among 320
Enumerating clusters: 120 among 320
Enumerating clusters: 140 among 320
Enumerating clusters: 160 among 320
Enumerating clusters: 180 among 320
Enumerating clusters: 200 among 320
Enumerating clusters: 220 among 320
Enumerating clusters: 240 among 320
Enumerating clusters: 260 among 320
Enumerating clusters: 280 among 320
Enumerating clusters: 300 among 320
Enumerating clusters: 320 among 320
Done imputation. It took 0.03 mins


In [ ]:
# We will keep StructureType we just imputed

Infer_features_for_HazusDL = importer.get_class("HazusInfererEarthquake")
inferer=Infer_features_for_HazusDL(input_inventory=hazus_inferred_inventory_imputed, 
                                n_possible_worlds=NO_POSSIBLE_WORLDS, 
                                yearBuilt_key = 'erabuilt', 
                                structureType_key = 'StructureType',
                                clean_features= False)
hazus_inventory_final = inferer.infer()

>> Step1 : Checking if OccupancyClass (OccupancyClass) exist.
>> Step2-1 : Checking if StructureType (StructureType) and ReplacementCost (ReplacementCost) exist
>> Step3-1 : Checking if BuildingRise (BuildingRise), DesignLevel (DesignLevel) and FoundationType (FoundationType) exist
>> Step3-2 : Inferring {'DesignLevel'}
>> Step4 : Changing feature names to what R2D (pelicun) can recognize
Done inference. It took 0.27 mins


# Change Attribute Names To Make Them Compatible with R2D

In [14]:
hazus_inventory_final.change_feature_names({'erabuilt': 'YearBuilt',
                                            'lat': 'Latitude',
                                            'lon': 'Longitude',
                                            'fpAreas': 'PlanArea',
                                            'numstories': 'NumberOfStories'})
counter = 0
for _, val in hazus_inventory_final.inventory.items():
    val.add_features({"id": counter})
    counter += 1

# Write the Created Inventory in a GeoJSON File

In [15]:
_ = hazus_inventory_final.write_to_geojson(
    output_file=INVENTORY_OUTPUT)